Notebook with toy example to test/demonstrate bayesian hierarchical clustering

In [1]:
import scgenome.simulation as sim
import numpy as np
import pandas as pd
from IPython.display import display
from scgenome import cncluster
from scgenome.utils import cn_mat_to_cn_data, cn_mat_as_df

NUM_SAMPLE = 5
NUM_BIN = 12
CHR_NAMES = ["1","2"]

#max_cn = cn_mat.max()
max_cn = 7
print(f"max_cn: {max_cn}")

cluster1 = sim.cn_mat_poisson(NUM_SAMPLE, NUM_BIN, init_lambda=1., jump_lambda=1., seed=1, max_cn=max_cn)
cluster2 = sim.cn_mat_poisson(NUM_SAMPLE, NUM_BIN, init_lambda=3., jump_lambda=3., seed=3, max_cn=max_cn)

clst1_cell_ids = [f"cl1_cell{i}" for i in range(cluster1.shape[0])]
clst2_cell_ids = [f"cl2_cell{i}" for i in range(cluster1.shape[0])]

cn_mat = np.concatenate([cluster1, cluster2])
cell_ids = clst1_cell_ids + clst2_cell_ids


df_cn_mat = cn_mat_as_df(cn_mat,CHR_NAMES)
cn_data = cn_mat_to_cn_data(df_cn_mat, cell_id_vals=cell_ids)
cn_data["cluster_id"] = cn_data["cell_id"].str.split("_",expand=True).iloc[:,0]


cn_data["copy2"] = cn_data["copy"] + np.absolute(np.random.normal(size=cn_data.shape[0], scale=0.3))
cn_data.columns = ["chr", "bin", "cell_id", "state", "start", "end", "cluster_id", "copy"]

display(cn_data.head())

cncluster.bayesian_cluster(cn_data, n_states=max_cn, value_ids=["copy"])

max_cn: 7


,chr,bin,cell_id,state,start,end,cluster_id,copy
0,1,0,cl1_cell0,1,0,9,cl1,1.417114
1,1,1,cl1_cell0,0,10,19,cl1,0.276803
2,1,2,cl1_cell0,0,20,29,cl1,0.252216
3,1,3,cl1_cell0,0,30,39,cl1,0.325670
4,1,4,cl1_cell0,0,40,49,cl1,0.295776


ValueError: operands could not be broadcast together with shapes (1,2,1,12) (1,2,2) 